<a href="https://colab.research.google.com/github/niikun/DL_for_health_care/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab1
この最初のラボでは、Tensors、Loss、Autogradを含むPyTorchの基礎を紹介します。うまくいけば、PyTorchに慣れ始めるでしょう。最後に、いくつかの合成データを使ってsratchから線形回帰モデルを実装します。
Table of Contents
- Tensors
- Loss
- Autograd
- Assignment


In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
#set seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"]=str(seed)

## Tensor

### 1.1 Initializing Tensor

- Directly from data

In [3]:
data = [[1,2],[3,4]]
x_data = torch.tensor(data)
x_data

tensor([[1, 2],
        [3, 4]])

- from a numpy array

In [4]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
x_np

tensor([[1, 2],
        [3, 4]])

- from another tensor

In [5]:
x_ones = torch.ones_like(x_data)
print(f"ones tensor :\n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float)
print(f"random tensor :\n {x_rand} \n")

ones tensor :
 tensor([[1, 1],
        [1, 1]]) 

random tensor :
 tensor([[0.8823, 0.9150],
        [0.3829, 0.9593]]) 



- with random or constant values:

In [6]:
shape = (2,3)
rand_tensor=torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"random tensor :\n {rand_tensor} \n")
print(f"ones tensor :\n {ones_tensor} \n")
print(f"zeros tensor \n {zeros_tensor}\n")

random tensor :
 tensor([[0.3904, 0.6009, 0.2566],
        [0.7936, 0.9408, 0.1332]]) 

ones tensor :
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

zeros tensor 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])



### 1.2 Attributes of a tensor

In [7]:
tensor = torch.rand(3,4)
print(f"shape of tensor : {tensor.shape}")
print(f"data type of tensor : {tensor.dtype}")
print(f"device tensor is stored on : {tensor.device}")

shape of tensor : torch.Size([3, 4])
data type of tensor : torch.float32
device tensor is stored on : cpu


### 1.3 operations on tensors

- Standard numpy-like indexing and slicing:

In [8]:
tensor = torch.arange(12).reshape(3,4).float()
print(tensor)
print("first row",tensor[0])
print("first columns",tensor[:,0])
print("last columns",tensor[:,-1])

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])
first row tensor([0., 1., 2., 3.])
first columns tensor([0., 4., 8.])
last columns tensor([ 3.,  7., 11.])


- Joining tensors

In [9]:
t1 = torch.cat([tensor,tensor,tensor],dim=1)
t2 = torch.cat([tensor,tensor,tensor],dim=0)
print(t1)
print(t2)

tensor([[ 0.,  1.,  2.,  3.,  0.,  1.,  2.,  3.,  0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.,  4.,  5.,  6.,  7.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.,  8.,  9., 10., 11.,  8.,  9., 10., 11.]])
tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])


- Arithmetic operations

In [10]:
y1 = tensor @ tensor.T
print(f"y1 : \n{y1} \n")

y2 = tensor.matmul(tensor.T)
print(f"y2 : \n{y2} \n")

y3 = torch.rand_like(tensor)
torch.matmul(tensor,tensor.T,out=y3)
print(f"y3 : \n{y3} \n")

y1 : 
tensor([[ 14.,  38.,  62.],
        [ 38., 126., 214.],
        [ 62., 214., 366.]]) 

y2 : 
tensor([[ 14.,  38.,  62.],
        [ 38., 126., 214.],
        [ 62., 214., 366.]]) 

y3 : 
tensor([[ 14.,  38.,  62.],
        [ 38., 126., 214.],
        [ 62., 214., 366.]]) 



<ipython-input-10-cdd0d0af78a1>:8: UserWarning: An output with one or more elements was resized since it had shape [3, 4], which does not match the required output shape [3, 3]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  torch.matmul(tensor,tensor.T,out=y3)


In [11]:
# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
print(z1)

z2 = tensor.mul(tensor)
print(z2)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)
print(z3)

tensor([[  0.,   1.,   4.,   9.],
        [ 16.,  25.,  36.,  49.],
        [ 64.,  81., 100., 121.]])
tensor([[  0.,   1.,   4.,   9.],
        [ 16.,  25.,  36.,  49.],
        [ 64.,  81., 100., 121.]])
tensor([[  0.,   1.,   4.,   9.],
        [ 16.,  25.,  36.,  49.],
        [ 64.,  81., 100., 121.]])


### 1.4 GPU Acceleration

In [12]:
mat = torch.rand(5000,5000)
print(mat.shape)
mat

torch.Size([5000, 5000])


tensor([[0.7886, 0.5895, 0.7539,  ..., 0.9313, 0.6453, 0.9844],
        [0.8312, 0.8682, 0.9359,  ..., 0.4046, 0.9987, 0.8608],
        [0.1268, 0.2253, 0.1223,  ..., 0.3939, 0.4493, 0.5327],
        ...,
        [0.5851, 0.5824, 0.8857,  ..., 0.3165, 0.4845, 0.3896],
        [0.3348, 0.1535, 0.5840,  ..., 0.0285, 0.7444, 0.5193],
        [0.4027, 0.7190, 0.3847,  ..., 0.9606, 0.6629, 0.0359]])

In [13]:
%%time
torch.mm(mat.t(),mat)

CPU times: user 4.84 s, sys: 114 ms, total: 4.95 s
Wall time: 5.33 s


tensor([[1697.6150, 1260.7886, 1260.5798,  ..., 1276.7966, 1265.4301,
         1273.4854],
        [1260.7886, 1655.7583, 1247.6844,  ..., 1256.2369, 1263.6820,
         1263.9419],
        [1260.5798, 1247.6844, 1658.2700,  ..., 1258.1453, 1267.2045,
         1274.8801],
        ...,
        [1276.7966, 1256.2367, 1258.1453,  ..., 1704.4204, 1270.7224,
         1275.9523],
        [1265.4302, 1263.6820, 1267.2046,  ..., 1270.7224, 1686.0026,
         1272.0712],
        [1273.4852, 1263.9419, 1274.8801,  ..., 1275.9523, 1272.0712,
         1699.9994]])

In [14]:
%%time
if torch.cuda.is_available():
    mat = mat.cuda()
    torch.mm(mat.t(),mat)
else:
    print("no cuda")

CPU times: user 94.2 ms, sys: 230 ms, total: 324 ms
Wall time: 489 ms


### Exercise 1
$$
 \sigma(x) = \frac {1}{1+exp(-x)}
$$

In [14]:
def simoid(x):
    return 1/(1+torch.exp(-x))

### Exercise 2

$$
softmax(X)_{ij} = \frac{exp(X_{ij})}{\sum_kexp(X_{ik})}
$$

In [14]:
def softmax(X):
    return torch.exp(X)/torch.sum(torch.exp(X),dim=1).unsqueeze(1)


In [15]:
X = torch.tensor([[0.2288, 0.4111, 0.0385], [0.6233, 0.0364, 0.1999]])

In [20]:
torch.exp(X)

tensor([[1.2571, 1.5085, 1.0393],
        [1.8651, 1.0371, 1.2213]])

In [19]:
torch.sum(torch.exp(X),dim=1)

tensor([3.8048, 4.1234])

Tensor配列のサイズ1の次元を削除する: torch.squeeze

 指定した位置にサイズ1の次元を挿入する: torch.unsqueeze

In [21]:
torch.sum(torch.exp(X),dim=1).unsqueeze(1)

tensor([[3.8048],
        [4.1234]])

### Exercise 3
$$
O = XW +b
$$

In [22]:
def linear(X, W, b):
    return X @ W + b

In [23]:
X = torch.Tensor([[0.1, 0.2, 0.3]])
W = torch.Tensor([[0.1, 0.2, 0.3]]).T
b = torch.Tensor([-0.5])
assert torch.allclose(linear(X, W, b), torch.Tensor([[-0.3600]]), rtol=1e-2)

## Loss

学習データを提示された場合、未学習のネットワークは正しい答えを返さない可能性が高い。  
損失関数は、得られた結果の目標値に対する非類似度を測定するもので、学習中に最小化したい損失関数である。  
損失を計算するには、与えられたデータサンプルの入力を使って予測を行い、真のデータラベル値と比較する。

一般的な損失関数には、回帰タスク用のnn.MSELoss (Mean Square Error)と分類用のnn.NLLoss (Negative Log Likelihood)があります。  
nn.CrossEntropyLossはnn.LogSoftmaxとnn.NLLossを組み合わせたものです。  
nn.BCELossはバイナリ分類用に特別に設計されています。

- mean squared error
$$
L^{(i)} = \frac{1}{2} (\hat{y}^{(i)}-y^{(i)})^2
$$


In [24]:
def squared_loss(y_hat, y):
    return ((y_hat - y.reshape(y_hat.shape)) ** 2 / 2).mean()

### Exercise 4
$$
i^{(i)} = -\sum_{i=1}y^{(i)}_j\log\hat{y}^{(i)},  
$$
$$
L = \frac {1}{N}\sum_{i=1}^ni^{(i)}
$$

In [25]:
y = torch.tensor([[1, 0, 0], [0, 0, 1]])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])

In [30]:
torch.sum(-torch.sum(y * torch.log(y_hat),dim=1))/y.shape[0]

tensor(1.4979)

In [ ]:
def cross_entropy(y_hat, y):
    return -torch.sum(y * torch.log(y_hat))/y.shape[0]

## Autograd

## Assiginment